## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
#matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Poya,NC,-21.3500,165.1500,70.18,81,99,6.33,overcast clouds,2022-04-27 16:51:21
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,20,17.27,few clouds,2022-04-27 16:50:04
2,2,Rio Gallegos,AR,-51.6226,-69.2181,42.85,70,40,21.85,scattered clouds,2022-04-27 16:47:07
3,3,Port Alfred,ZA,-33.5906,26.8910,62.19,82,1,14.92,clear sky,2022-04-27 16:51:36
4,4,Bluff,NZ,-46.6000,168.3333,49.68,84,63,5.95,broken clouds,2022-04-27 16:46:24


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,87.76,58,20,17.27,few clouds,2022-04-27 16:50:04
6,6,Miram Shah,PK,33.0086,70.0697,82.11,20,44,1.23,scattered clouds,2022-04-27 16:51:37
7,7,Rikitea,PF,-23.1203,-134.9692,77.67,72,51,24.00,broken clouds,2022-04-27 16:48:50
20,20,Bonthe,SL,7.5264,-12.5050,84.29,68,82,11.05,broken clouds,2022-04-27 16:51:40
23,23,Teknaf,BD,20.8624,92.3058,78.28,92,16,3.58,few clouds,2022-04-27 16:51:40
26,26,Mahebourg,MU,-20.4081,57.7000,75.49,88,40,6.91,scattered clouds,2022-04-27 16:51:41
30,30,Hargeysa,SO,9.5600,44.0650,77.49,53,57,3.20,broken clouds,2022-04-27 16:51:42
34,34,Castro,BR,-24.7911,-50.0119,78.71,50,3,8.86,clear sky,2022-04-27 16:48:59
37,37,Kavieng,PG,-2.5744,150.7967,82.96,78,36,15.12,scattered clouds,2022-04-27 16:51:44
40,40,Sai Buri,TH,6.7013,101.6168,79.84,84,76,4.59,broken clouds,2022-04-27 16:51:45


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

236


City_ID                236
City                   236
Country                233
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
Date                   236
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")

In [7]:
#check if null values were removed
#clean_df.isnull().sum()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,87.76,few clouds,-22.9661,-42.0278,
6,Miram Shah,PK,82.11,scattered clouds,33.0086,70.0697,
7,Rikitea,PF,77.67,broken clouds,-23.1203,-134.9692,
20,Bonthe,SL,84.29,broken clouds,7.5264,-12.5050,
23,Teknaf,BD,78.28,few clouds,20.8624,92.3058,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,87.76,few clouds,-22.9661,-42.0278,Pousada Porto Praia
6,Miram Shah,PK,82.11,scattered clouds,33.0086,70.0697,Bachelor Hostel
7,Rikitea,PF,77.67,broken clouds,-23.1203,-134.9692,People ThankYou
20,Bonthe,SL,84.29,broken clouds,7.5264,-12.5050,Bonthe Holiday Village
23,Teknaf,BD,78.28,few clouds,20.8624,92.3058,Forestry Guesthouse


In [11]:
print(len(hotel_df))
hotel_df.count()

233


City                   233
Country                233
Max Temp               233
Current Description    233
Lat                    233
Lng                    233
Hotel Name             233
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
print(len(clean_hotel_df))
clean_hotel_df.count()

233


City                   233
Country                233
Max Temp               233
Current Description    233
Lat                    233
Lng                    233
Hotel Name             233
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/City_Data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))